In [4]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

In [2]:
client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

839
3600


In [3]:
#create a champ convert dictionary to get champ idx from id, idx range from [0, 133)
#also provide functions to get champ names from id
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

https://global.api.pvp.net/api/lol/static-data/kr/v1.2/champion?champData=all&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://global.api.pvp.net/api/lol/static-data/kr/v1.2/item?itemListData=all&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24


In [8]:
labels = np.fromfile('cluster_labels.txt', dtype=int, count=-1, sep=" ")

In [12]:
for cluster_idx in range(15):
    print cluster_idx
    for champ_idx,a_idx in enumerate(labels):
        if a_idx == cluster_idx:
            champ_id = GetIdFromIdx(champ_idx)
            champ_name = GetNameFromId(champ_id)
            print champ_name

0
Shaco
Nidalee
Karthus
Shyvana
Kindred
Nocturne
Vi
1
Zyra
Braum
MissFortune
Blitzcrank
Maokai
Thresh
Karma
Bard
Morgana
Ivern
Leona
Alistar
TahmKench
Nami
Soraka
Janna
Nautilus
2
Corki
Xerath
Orianna
Fizz
Kassadin
Viktor
Cassiopeia
Syndra
Ryze
TwistedFate
Taliyah
Velkoz
Varus
Katarina
Ekko
AurelionSol
Ahri
Malzahar
Lissandra
Zed
3
Brand
Trundle
Sona
Annie
Shen
Zac
Sion
Zilean
Nunu
Gragas
Lulu
Taric
4
Sivir
Lucian
Jinx
Kalista
Talon
Vayne
Caitlyn
Ashe
Twitch
Draven
Jhin
Ezreal
5
MonkeyKing
Warwick
Illaoi
Tryndamere
Yorick
DrMundo
Volibear
Sejuani
Teemo
Urgot
Galio
Heimerdinger
Skarner
Garen
MasterYi
Aatrox
FiddleSticks
XinZhao
Tristana
Veigar
Quinn
6
Rammus
Malphite
Lux
Vladimir
Kayle
Ziggs
Swain
Nasus
Chogath
Udyr
Azir
Amumu
Anivia
Singed
Mordekaiser
KogMaw
Diana
JarvanIV
7
Jayce
Graves
Hecarim
Khazix
Rengar
Rumble
Elise
LeeSin
RekSai
Evelynn
Leblanc
8
Jax
Kled
Darius
Riven
Gangplank
Poppy
Gnar
Renekton
Fiora
Olaf
Akali
Yasuo
Kennen
Pantheon
Irelia
9
10
11
12
13
14


In [132]:
match_dict = {}

for a_result in match_results:
    if a_result['combo'] in match_dict.keys():
        match_dict[a_result['combo']].append(a_result['result'])
    else:
        match_dict[a_result['combo']] = []
        match_dict[a_result['combo']].append(a_result['result'])

        #print a_result['result']
print len(match_results)
print len(match_dict)

max_count = 0
max_key = ''

for a_sum in match_dict:
    if len(match_dict[a_sum]) > 8:
        result_list = match_dict[a_sum]
        res_sum = sum(result_list)
        res_len = len(result_list)
        print a_sum
        print 'win %d in %d percent - %d' % (res_sum, res_len, (res_sum * 100)/(float)(res_len))

2590
1411
[ 8.  0.  2.  4.  1.  1.  7.  2.  4.  1.]
win 5 in 9 percent - 55
[ 8.  0.  2.  4.  1.  8.  7.  2.  4.  1.]
win 8 in 14 percent - 57
[ 7.  7.  2.  4.  1.  7.  7.  2.  4.  1.]
win 6 in 11 percent - 54
[ 8.  7.  6.  4.  1.  8.  7.  2.  4.  1.]
win 8 in 10 percent - 80
[ 8.  7.  2.  4.  3.  8.  7.  2.  4.  1.]
win 6 in 11 percent - 54
[ 1.  7.  2.  4.  1.  8.  7.  2.  4.  1.]
win 24 in 50 percent - 48
[ 6.  7.  2.  4.  1.  8.  7.  2.  4.  1.]
win 6 in 12 percent - 50
[ 8.  7.  2.  4.  1.  8.  7.  7.  4.  1.]
win 8 in 18 percent - 44
[ 1.  7.  2.  4.  1.  2.  7.  2.  4.  1.]
win 4 in 11 percent - 36
[ 8.  7.  2.  4.  1.  8.  7.  2.  4.  3.]
win 6 in 10 percent - 60
[ 8.  7.  2.  4.  1.  8.  7.  2.  4.  1.]
win 76 in 158 percent - 48
[ 7.  7.  2.  4.  1.  8.  7.  2.  4.  1.]
win 43 in 77 percent - 55
[ 8.  7.  2.  4.  1.  3.  7.  2.  4.  1.]
win 10 in 21 percent - 47
[ 7.  7.  2.  4.  3.  8.  7.  2.  4.  1.]
win 3 in 9 percent - 33
[ 7.  7.  2.  4.  1.  1.  7.  2.  4.  1.]
win 6 i

In [118]:
#load a match, print out its champion selections and win result

CLUSTER_NUM = 9
MATCH_NUM = matches_collection.count()
features = np.zeros((MATCH_NUM, CLUSTER_NUM * 2))
targets = np.zeros(MATCH_NUM)
lane_dict = {'TOP':0, 'JUNGLE':1, 'MIDDLE':2, 'BOTTOM':3}
role_dict = {'NONE':0, 'SOLO':1, 'DUO_CARRY':2, 'DUO_SUPPORT':3}
#reorder with TOP_SOLO JUNGLE_NONE MIDDLE_SOLO BOTTOM_DUO_CARRY BOTTOM_DUO_SUPPORT
match_results = []
valid_idx = 0
cluster_combination_dict = {}
for a_idx, a_match in enumerate(matches_collection.find()):
    is_match_valid = True
    a_team = []
    a_duration = a_match['matchDuration']
    a_minute = a_duration/60
    reordered_champ_idxs = np.zeros((10))
    reordered_champ_clusters = np.zeros((10))
    if a_minute <= 20:
        is_match_valid = False
        #print 'EXCEPTION DURATION ' + str(a_idx)
    
    #print '---BEGIN---'
    for part_idx, a_part in enumerate(a_match['participants']):
        champ_id = a_part['championId']
        champ_idx = GetIdxFromId(champ_id)
        a_team.append(labels[champ_idx])
        champ_lane = a_part['timeline']['lane']
        champ_role = a_part['timeline']['role']
        #print GetNameFromId(champ_id)
        #print champ_lane
        #print champ_role
        isValidLane = champ_lane in lane_dict.keys()
        isValidRole = champ_role in role_dict.keys()
        if isValidLane == False:
            is_match_valid = False
            #print "EXCEPTION LANE " + str(a_idx)
        if isValidRole == False:
            is_match_valid = False
            #print "EXCEPTION ROLE " + str(a_idx)
        
        team_offset = 0
        if part_idx >= 5:
            team_offset = 5
        if champ_lane == 'TOP' and champ_role == 'SOLO':
            reordered_champ_idxs[0 + team_offset] = champ_idx
        if champ_lane == 'JUNGLE' and champ_role == 'NONE':
            reordered_champ_idxs[1 + team_offset] = champ_idx        
        if champ_lane == 'MIDDLE' and champ_role == 'SOLO':
            reordered_champ_idxs[2 + team_offset] = champ_idx
        if champ_lane == 'BOTTOM' and champ_role == 'DUO_CARRY':
            reordered_champ_idxs[3 + team_offset] = champ_idx
        if champ_lane == 'BOTTOM' and champ_role == 'DUO_SUPPORT':
            reordered_champ_idxs[4 + team_offset] = champ_idx 
            
    for i in range(10):
        reordered_champ_clusters[i] = labels[reordered_champ_idxs[i]]
            
    #print reordered_champ_clusters
            
    #print '---END---'
    a_team = np.array(a_team)
    #print '%d %d %d %d %d vs %d %d %d %d %d' % (a_team[0],a_team[1],a_team[2],a_team[3],a_team[4],a_team[5],a_team[6],a_team[7],a_team[8],a_team[9])
    a_feature_blue = []
    a_feature_red = []
    for i in range(CLUSTER_NUM):
        a_feature_blue.append(sum(a_team[0:5] == i))
        a_feature_red.append(sum(a_team[5:10] == i))
    #print a_feature_blue
    #print a_feature_red
    a_feature = a_feature_blue + a_feature_red
    #0 if blue win, 1 if red win
    a_target = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        a_target = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        a_target = 1
    #print a_target
    #print a_feature
    #print a_target
    if is_match_valid == True:
        features[valid_idx, :] = a_feature
        targets[valid_idx] = a_target
        valid_idx += 1
        match_results.append({'combo':str(reordered_champ_clusters), 'result':a_target})
    
    if False:#a_idx == 10:
        print valid_idx
        break
    if a_idx%100 == 0:
        #break
        print a_idx

print features
print targets
print match_results

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:58: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
[[ 0.  2.  0. ...,  0.  1.  0.]
 [ 0.  1.  1. ...,  0.  1.  0.]
 [ 0.  1.  1. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[ 0.  0.  0. ...,  0.  0.  0.]
[{'result': 0, 'combo': '[ 1.  7.  7.  4.  1.  1.  7.  2.  4.  1.]'}, {'result': 0, 'combo': '[ 3.  5.  2.  4.  1.  1.  7.  1.  4.  1.]'}, {'result': 0, 'combo': '[ 8.  2.  5.  4.  1.  1.  7.  2.  4.  1.]'}, {'result': 1, 'combo': '[ 8.  7.  6.  4.  1.  8.  7.  2.  4.  1.]'}, {'result': 1, 'combo': '[ 1.  7.  2.  4.  1.  8.  7.  2.  4.  1.]'}, {'result': 0, 'combo': '[ 1.  7.  5.  4.  1.  7.  7.  2.  4.  1.]'}, {'result': 0, 'combo': '[ 2.  7.  2.  4.  1.  8.  7.  2.  2.  1.]'}, {'result': 1, 'combo': '[ 8.  7.  2.  4.  1.  2.  7.  2.  4.  1.]'}, {'result': 0, 'combo': '[ 1.  7.  2.  4.  1.  8.

In [98]:
from sklearn import svm

clf = svm.SVC()
clf.fit(features_training, targets_training)
predictions_training = clf.predict(features_training)
predictions_testing = clf.predict(features_testing)

In [57]:
print sum(predictions_training == targets_training)/(float)(len(targets_training))
print sum(predictions_testing == targets_testing)/(float)(len(targets_testing))
print len(targets)

0.58
0.531666666667
3600


In [63]:
from sklearn.neural_network import MLPClassifier

features_training = features[0:3000, :]
features_testing = features[3000:3600, :]
targets_training = targets[0:3000]
targets_testing = targets[3000:3600]

mlp = MLPClassifier(hidden_layer_sizes=(10, 3), max_iter=1000, alpha=1e-5,
                    solver='lbfgs', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

mlp.fit(features_training, targets_training)
print("Training set score: %f" % mlp.score(features_training, targets_training))
print("Test set score: %f" % mlp.score(features_testing, targets_testing))

Training set score: 0.587667
Test set score: 0.495000


In [67]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth = 5, n_estimators=10, max_features=1)
clf.fit(features_training, targets_training)
score = clf.score(features_testing, targets_testing)
print score

0.52
